# Sushi Macaque - flip the coin strategy

An example how to randomly go all-in to a new token every day.

* The prerequisites for the token is that we have not bought it before

* The token most have USD 500k+ volume before it can be chosen

* This is a simplified example strategy that ignores available liquidity and loss of trade balance due to slippage

* The skeleton of this strategy is based on [Teddy Koker's momentum strategy example](https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/)

## Creating trading universe

First let's import libraries and initialise our dataset client.

In [5]:
from random import Random

import pandas as pd

from capitalgram.chain import ChainId
from capitalgram.pair import PairUniverse, PandasPairUniverse

try:
    import capitalgram
except ImportError:
    !pip install -e git+https://github.com/miohtama/capitalgram-onchain-dex-quant-data.git#egg=capitalgram
    import site
    site.main()

from capitalgram.client import Capitalgram

capitalgram = Capitalgram.create_jupyter_client()

Started Capitalgram in Jupyter notebook environment, configuration is stored in /Users/moo/.capitalgram


Let's create a pair universe for Sushi. [See full example](https://docs.capitalgram.com/examples/pairs.html).
We will create a dataset of 4h candles that trade on Sushiswap on Ethereum.

In [6]:
# Decompress the pair dataset to Python map
columnar_pair_table = capitalgram.fetch_pair_universe()

# Exchange map data is so small it does not need any decompression
exchange_universe = capitalgram.fetch_exchange_universe()

# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
sushi_swap = exchange_universe.get_by_name_and_chain(ChainId.ethereum, "sushiswap")
sushi_pairs: pd.DataFrame = all_pairs_dataframe.loc[all_pairs_dataframe['exchange_id'] == sushi_swap.exchange_id]

# Create a Python set of pair ids
wanted_pair_ids = sushi_pairs["pair_id"]

print(f"Sushiswap on Ethereum has {len(wanted_pair_ids)} pairs")

Sushiswap on Ethereum has 1308 pairs


Get daily candles and filter them against our wanted pair set.

In [7]:
from capitalgram.candle import CandleBucket, GroupedCandleUniverse
from capitalgram.pair import PandasPairUniverse
from capitalgram.frameworks.backtrader import prepare_candles_for_backtrader

from pandas.core.groupby import GroupBy

# Get daily candles as Pandas DataFrame
all_candles = capitalgram.fetch_all_candles(CandleBucket.h24).to_pandas()
sushi_candles: pd.DataFrame = all_candles.loc[all_candles["pair_id"].isin(wanted_pair_ids)]

sushi_candles = prepare_candles_for_backtrader(sushi_candles)

# Make the trading pair data easily accessible
pair_universe = PandasPairUniverse(sushi_pairs)

# Group candles by the trading pair ticker
sushi_tickers = GroupedCandleUniverse(sushi_candles)

print(f"Out candle universe size is {len(sushi_candles)}")


Out candle universe size is 58627


## Creating coin flip backtrader strategy

[See the Backtrader quickstart tutorial](https://www.backtrader.com/docu/quickstart/quickstart/).

In [8]:
import backtrader as bt
from backtrader.indicators import PeriodN


class PastTradeVolumeIndicator(PeriodN):
    """Indicates whether the trading pair has reached certain volume for the last N days.

    Based on indicator base class that takes period (days) as an input.

    https://github.com/mementum/backtrader/blob/0fa63ef4a35dc53cc7320813f8b15480c8f85517/backtrader/indicators/basicops.py#L33
    """

    lines = ('cum_volume',)

    def __init__(self):
        import ipdb ; ipdb.set_trace()
        self.lines.cum_volume = bt.Max(0.0, self.params.value)

    def next(self):
        self.lines.dummyline[0] = max(0.0, self.params.value)


class SushiMacaqueStrategy(bt.Strategy):
    """A strategy that picks a new token every day."""

    def __init__(self, pair_universe: PandasPairUniverse, seed: int):
        #: Allows us to print human-readable pair information
        self.pair_universe = pair_universe

        #: Initialize (somewhat) determininistic random number generator
        self.dice = Random(seed)

        #: We operate on daily candles.
        #: At each tick, we process to the next candle
        self.day = 0

        self.indicators = {}

        for pair_candles in self.data:
            self.indicators[pair_candles] = PastTradeVolumeIndicator(pair_candles["volume"])

    def next(self):
        # Simply log the closing price of the series from the reference
        self.day += 1
        print("Hello")

## Feed the strategy

Feed in Sushiswap data to the backtrader strategy

In [9]:
# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)

# Add a strategy
cerebro.addstrategy(SushiMacaqueStrategy, pair_universe=pair_universe, seed=0x1000)

# Pass all Sushi pairs to the data fees to the strategy
for df in sushi_tickers.get_all_candle_fees():
    backtrader_feed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(backtrader_feed)


AttributeError: 'GroupedCandleUniverse' object has no attribute 'get_all_candle_fees'

## Running the strategy

In [ ]:
# Run over everything
print("Running")
cerebro.run()
print("Done")

## Plotting the results

In [ ]:
# TODO: Displaying graphics from Backtrader in Jupyter notebook is broken
# See  https://github.com/enzoampil/fastquant/issues/382
#
# Returns two figures
# figures = cerebro.plot()
# figures[0][0]
